
<a id='classical-filtering'></a>
<div id="qe-notebook-header" style="text-align:right;">
        <a href="https://quantecon.org/" title="quantecon.org">
                <img style="width:250px;display:inline;" src="https://assets.quantecon.org/img/qe-menubar-logo.svg" alt="QuantEcon">
        </a>
</div>

# Filtragem Clássica com Álgebra Linear

## Conteúdo

- [Filtragem Clássica com Álgebra Linear](#Filtragem-Clássica-com-Álgebra-Linear)  
  - [Resumo](#Resumo)  
  - [Predição de Horizontes Infinitos e Problemas de Filtragem](#Predição-de-Horizontes-Infinitos-e-Problemas-de-Filtragem)  
  - [Predição de Dimensões Finitas](#Predição-de-Dimensões-Finitas)  
  - [Controle Dimensional Finito Combinado e Previsão](#Controle-Dimensional-Finito-Combinado-e-Previsão)  
  - [Exercícios](#Exercícios)  

> *Devidamente traduzido, revisado e adaptado do [QuantEcon](https://quantecon.org/) pelos bolsistas CNPq, Pedro Luiz H. Furtado e Jonas Aragão M. Corpes, sob supervisão do Prof. Christiano Penna, do CAEN/UFC.*

## Resumo

Esta é uma sequência da aula anterior [Controle clássico com álgebra linear](https://julia.quantecon.org/.html).

Essa aula usou álgebra linear - em particular, a [decomposição LU](https://en.wikipedia.org/wiki/LU_decomposition) - para formular e resolver uma classe de problemas de controle ótimo quadrático-linear.

Nesta aula, usaremos uma decomposição intimamente relacionada, a [decomposição de Cholesky](https://en.wikipedia.org/wiki/Cholesky_decomposition), para resolver problemas de previsão e filtragem linear.

Exploramos o fato útil de que há uma conexão íntima entre duas classes de problemas superficialmente diferentes:

- problemas ótimos de controle determinístico quadrático-linear (QL)
- previsão de mínimos quadrados lineares e problemas de filtragem


A primeira classe de problemas não envolve aleatoriedade, enquanto a segunda é sobre aleatoriedade.

No entanto, essencialmente a mesma matemática resolve ambos os tipos de problemas.

Essa conexão, que é frequentemente denominada "dualidade", está presente, independentemente de se usar procedimentos de solução "clássicos" ou "recursivos".

De fato, vimos a dualidade em ação anteriormente, quando formulamos recursivamente problemas de controle e previsão em aulas sobre [problemas de programação dinâmica do LQ](https://julia.quantecon.org/../dynamic_programming/lqcontrol.html), [Uma primeira olhada no Filtro de Kalman](https://julia.quantecon.org/../tools_and_techniques/kalman.html) e [O modelo de renda permanente](https://julia.quantecon.org/../dynamic_programming/perm_income.html).

Uma consequência útil da dualidade é que:

- Com todo problema de controle QL, é associado implicitamente um problema de previsão ou filtragem linear dos mínimos quadrados.
- Com todos os problemas de previsão ou filtragem linear dos mínimos quadrados, há implicitamente associado um problema de controle de QL.


Um entendimento dessas conexões tem se mostrado útil repetidamente para resolver problemas aplicados interessantes.

Por exemplo, Sargent [Sar87](https://julia.quantecon.org/../zreferences.html#sargent1987) [chs. IX, XIV] e Hansen e Sargent [HS80](https://julia.quantecon.org/../zreferences.html#hansar1980) formularam e resolveram problemas de controle e filtragem usando $ z $ -métodos de transformação.

Nesta aula, investigamos essas idéias usando principalmente álgebra linear elementar.

### Referências

Referências úteis incluem [Whi63](https://julia.quantecon.org/../zreferences.html#whittle1963), [HS80](https://julia.quantecon.org/../zreferences.html#hansar1980), [Orf88](https://julia.quantecon.org/../zreferences.html#orfanidisoptimum1988), [AP91](https://julia.quantecon.org/../zreferences.html#athanasios1991) e [Mut60](https://julia.quantecon.org/../zreferences.html#muth1960).

### Configuração

In [1]:
using InstantiateFromURL
github_project("QuantEcon/quantecon-notebooks-julia", version = "0.5.0")
# github_project("QuantEcon/quantecon-notebooks-julia", version = "0.5.0", instantiate = true) # uncomment to force package installation

In [2]:
using LinearAlgebra, Statistics

## Predição de Horizontes Infinitos e Problemas de Filtragem

Colocamos dois problemas de previsão e filtragem relacionados.

Permitimos que $ Y_t $ seja uma média móvel univariada de $ m^{\rm ésima} $ ordem, processo estocástico estacionário de covariância:


<a id='equation-eq-24'></a>
$$
Y_t = d(L) u_t \tag{1}
$$

onde $ d (L) = \sum^m_{j = 0} d_j L^j $ e $ u_t $ é um processo aleatório estacionário não correlacionado em série que satisfaz:


<a id='equation-eq-25'></a>
$$
\begin{aligned}
    \mathbb{E} u_t &= 0\\
    \mathbb{E} u_t u_s &=
    \begin{cases}
        1 & \text{ se } t = s \\
        0 & \text{ caso contrário}
    \end{cases}
\end{aligned} \tag{2}
$$

Não impomos condições aos zeros de $ d (z) $.

Um segundo processo estacionário de covariância é $ X_t $ dado por:


<a id='equation-eq-26'></a>
$$
X_t = Y_t + \varepsilon_t \tag{3}
$$

em que $ \varepsilon_t $ é um material de escritório não correlacionado em série
processo aleatório com $ \mathbb {E} \varepsilon_t = 0 $ e $ \mathbb {E} \varepsilon_t \varepsilon_s $ = $ 0 $ para todos os $ t $ e $ s $ distintos.

Também assumimos que $ \mathbb {E} \varepsilon_t u_s = 0 $ para todos os $ t $ e $ s $.

O **problema de previsão de mínimos quadrados lineares** é encontrar o $ L_2 $ da
variável aleatória $ \hat X_ {t + j} $ entre combinações lineares de
$ \{X_t, \ X_ {t-1},
\ldots \} $ que minimiza $ \mathbb {E} (\hat X_ {t + j} - X_ {t + j}) ^ 2 $.

Ou seja, o problema é encontrar $ \gamma_j (L) = \sum ^ \infty_ {k = 0} \gamma_ {jk} \, L^k $ de modo que $ \sum^\infty_ {k = 0} \vert \gamma_ {jk} \vert ^ 2 <\infty $ e $ \mathbb {E} [\gamma_j \, (L) X_t -X_ {t + j}] ^ 2 $ é minimizado.

O **problema de filtragem de mínimos quadrados lineares** é encontrar um $ b \, (L) = \sum ^ \infty_ {j = 0} b_j \, L ^ j $ de modo que $ \sum ^ \infty_ {j = 0} \vert b_j \vert ^ 2 <\infty $ e $ \mathbb {E} [b \, (L) X_t-Y_t] ^ 2 $ são minimizados.

Versões interessantes desses problemas relacionados à teoria da renda permanente foram estudadas por [Mut60](https://julia.quantecon.org/../zreferences.html#muth1960).

### Formulação do problema

Esses problemas são resolvidos da seguinte maneira.

Os covariogramas de $ Y $ e $ X $ e seu covariograma cruzado são, respectivamente:


<a id='equation-eq-27'></a>
$$
\begin{aligned}
    C_X (\tau) &= \mathbb{E}X_t X_{t-\tau} \\
    C_Y (\tau) &= \mathbb{E}Y_t Y_{t-\tau}  \qquad \tau = 0, \pm 1, \pm 2, \ldots \\
    C_{Y,X} (\tau) &= \mathbb{E}Y_t X_{t-\tau}
\end{aligned} \tag{4}
$$

As funções geradoras de covariância e covariância cruzada são definidas como:


<a id='equation-eq-28'></a>
$$
\begin{aligned}
    g_X(z) &= \sum^\infty_{\tau = - \infty} C_X (\tau) z^\tau \\
    g_Y(z) &= \sum^\infty_{\tau = - \infty} C_Y (\tau) z^\tau \\
    g_{YX} (z) &= \sum^\infty_{\tau = - \infty} C_{YX} (\tau) z^\tau
\end{aligned} \tag{5}
$$

As funções geradoras podem ser calculadas usando os seguintes fatos.

Seja $ v_ {1t} $ e $ v_ {2t} $ sejam dois ruídos brancos não correlacionados e serialmente com variações de unidade.

Ou seja, $ \mathbb {E} v ^ 2_ {1t} = \mathbb {E} v ^ 2_ {2t} = 1, \mathbb {E} v_ {1t} = \mathbb {E} v_ {2t} = 0, \mathbb {E} v_ {1t} v_ {2s} = 0 $ para todos os $ t $ e $ s $, $ \mathbb {E} v_ {1t} v_ {1t-j} = \mathbb {E} v_ {2t} v_ {2t-j} = 0 $ para todos os $ j \not = 0 $.

Sejam $ x_t $ e $ y_t $ dois processos aleatórios dados por:

$$
\begin{aligned}
    y_t &= A(L) v_{1t} + B(L) v_{2t} \\
    x_t &= C(L) v_{1t} + D(L) v_{2t}
\end{aligned}
$$

Então, como mostrado, por exemplo, em [Sar87](https://julia.quantecon.org/../zreferences.html#sargent1987) [ch. XI], é verdade que:


<a id='equation-eq-29'></a>
$$
\begin{aligned}
    g_y(z) &= A(z) A(z^{-1}) + B (z) B(z^{-1}) \\
    g_x (z) &= C(z) C(z^{-1}) + D(z) D(z^{-1}) \\
    g_{yx} (z) &= A(z) C(z^{-1}) + B(z) D(z^{-1})
\end{aligned} \tag{6}
$$

Aplicando essas fórmulas a [(1)](#equation-eq-24) - [(4)](#equation-eq-27), temos


<a id='equation-eq-30'></a>
$$
\begin{aligned}
    g_Y(z) &= d(z)d(z^{-1}) \\
    g_X(z) &= d(z)d(z^{-1}) + h\\
    g_{YX} (z) &= d(z) d(z^{-1})
\end{aligned} \tag{7}
$$

O passo chave na obtenção de soluções para nossos problemas é fatorar a função geradora de covariância $ g_X (z) $ de $ X $.

As soluções de nossos problemas são dadas por fórmulas de Wiener e Kolmogorov.

Essas fórmulas utilizam a representação da média móvel Wold do processo $ X_t $,


<a id='equation-eq-31'></a>
$$
X_t = c\,(L)\,\eta_t \tag{8}
$$

onde $ c(L) = \sum^m_{j=0} c_j\, L^j $, com:


<a id='equation-eq-32'></a>
$$
c_0 \eta_t
= X_t - \mathbb{\hat E} [X_t \vert X_{t-1}, X_{t-2}, \ldots] \tag{9}
$$

Aqui $ \mathbb {\hat E} $ é o operador de projeção de mínimos quadrados lineares.

A equação [(9)](#equation-eq-32) é a condição em que $ c_0 \eta_t $ pode ser o erro de um passo à frente na previsão de $ X_t $ a partir de seus próprios valores passados.

A condição [(9)](#equation-eq-32) requer que $ \eta_t $ esteja no
espaço linear estendido por $ [X_t, \ X_ {t-1}, \ldots] $.

Isso será verdadeiro se, e somente se, os zeros de $ c (z) $ não estiverem dentro do círculo unitário.

É uma implicação de [(9)](#equation-eq-32) que $ \eta_t $ é uma série
processo aleatório não correlacionado e que uma normalização possa ser imposta
que $ \mathbb {E} \eta_t ^ 2 = 1 $.

Consequentemente, uma implicação de [(8)](#equation-eq-31) é
que a função geradora de covariância de $ X_t $ pode ser expressa
como:


<a id='equation-eq-33'></a>
$$
g_X(z) = c\,(z)\,c\,(z^{-1}) \tag{10}
$$

Resta discutir como $ c (L) $ deve ser calculado.

Combinando [(6)](#equation-eq-29) e [(10)](#equation-eq-33) fornece


<a id='equation-eq-34'></a>
$$
d(z) \,d(z^{-1}) + h = c \, (z) \,c\,(z^{-1}) \tag{11}
$$

Portanto, já mostramos construtivamente como fatorar a função geradora de covariância $ g_X (z) = d (z) \,d \, (z ^ {- 1}) + h $.

Agora, apresentamos o **operador de aniquilação**:


<a id='equation-eq-35'></a>
$$
\left[
    \sum^\infty_{j= - \infty} f_j\, L^j
\right]_+
\equiv \sum^\infty_{j=0} f_j\,L^j \tag{12}
$$

Em palavras, $ [\phantom {00}] _ + $ significa "ignorar potências negativas de $ L $".

Definimos a solução do problema de previsão como $ \mathbb {\hat E} [X_ {t + j} \vert X_t, \, X_ {t-1}, \ldots] = \gamma_j \, (L) X_t $.

Supondo que as raízes de $ c (z) = 0 $ estejam fora do círculo unitário, a fórmula de Wiener-Kolmogorov para $ \gamma_j (L) $ mantém:


<a id='equation-eq-36'></a>
$$
\gamma_j\, (L) =
\left[
    {c (L) \over L^j}
\right]_+ c\,(L)^{-1} \tag{13}
$$

Definimos a solução do problema de filtragem como $ \mathbb {\hat E} [Y_t \mid X_t, X_ {t-1}, \ldots] = b (L) X_t $.

A fórmula Wiener-Kolomogorov para $ b (L) $ é

$$
b(L) = \left({g_{YX} (L) \over c(L^{-1})}\right)_+ c(L)^{-1}
$$

or


<a id='equation-eq-37'></a>
$$
b(L) = \left[ {d(L)d(L^{-1}) \over c(L^{-1})} \right]_+ c(L)^{-1} \tag{14}
$$

As fórmulas [(13)](#equation-eq-36) e [(14)](#equation-eq-37) são discutidas em detalhes em [Whi83](https://julia.quantecon.org/../zreferences.html#whittle1983) e [Sar87](https://julia.quantecon.org/../zreferences.html#sargent1987).

O leitor interessado pode encontrar vários exemplos do uso dessas fórmulas em economia
Alguns exemplos clássicos de uso dessas fórmulas são devidos a [Mut60](https://julia.quantecon.org/../zreferences.html#muth1960).

Como exemplo da utilidade da fórmula [(14)](#equation-eq-37), permitimos que $ X_t $ seja um processo estocástico com representação da média móvel de Wold:

$$
X_t = c(L) \eta_t
$$

onde $ \mathbb {E} \eta ^ 2_t = 1, \hbox {e} c_0 \eta_t = X_t - \mathbb {\hat E} [X_t \vert X_ {t-1}, \ldots], c (L ) = \sum ^ m_ {j = 0} c_j L $.

Suponha que, no momento $ t $, desejamos prever uma soma geométrica dos futuros $ X $'s, a saber:

$$
y_t \equiv \sum^\infty_{j=0} \delta^j X_{t+j} = {1 \over 1 - \delta L^{-1}}
X_t
$$

dado conhecimento de $ X_t, X_ {t-1}, \ldots $.

Usaremos [(14)](#equation-eq-37) para obter a resposta.

Usando as fórmulas padrão [(6)](#equation-eq-29), temos que:

$$
\begin{aligned}
    g_{yx}(z) &= (1-\delta z^{-1})c(z) c (z^{-1}) \\
    g_x (z) &= c(z) c (z^{-1})
\end{aligned}
$$

Então [(14)](#equation-eq-37) se torna:


<a id='equation-eq-38'></a>
$$
b(L)=\left[{c(L)\over 1-\delta L^{-1}}\right]_+ c(L)^{-1} \tag{15}
$$

Para avaliar o termo no operador de aniquilação, usamos o seguinte resultado de [HS80](https://julia.quantecon.org/../zreferences.html#hansar1980).

**Proposição** 

Seja:

- $ g(z) = \sum^\infty_{j=0} g_j \, z^j $ onde $ \sum^\infty_{j=0} \vert g_j \vert^2 < + \infty $  
- $ h\,(z^{-1}) = $ $ (1- \delta_1 z^{-1}) \ldots (1-\delta_n z^{-1}) $, onde $ \vert \delta_j \vert < 1 $, para $ j = 1, \ldots, n $  


Então


<a id='equation-eq-39'></a>
$$
\left[{g(z)\over h(z^{-1})}\right]_+ = {g(z)\over h(z^{-1})} - \sum^n_{j=1}
\ {\delta_j g (\delta_j) \over \prod^n_{k=1 \atop k \not = j} (\delta_j -
\delta_k)} \ \left({1 \over z- \delta_j}\right) \tag{16}
$$

e, alternativamente,


<a id='equation-eq-40'></a>
$$
\left[
    {g(z)\over h(z^{-1})}
\right]_+
=\sum^n_{j=1} B_j
\left(
    {zg(z)-\delta_j g (\delta_j) \over z- \delta_j}
\right) \tag{17}
$$

onde $ B_j = 1 / \prod ^ n_ {k = 1 \ no topo de k + j} (1 - \delta_k / \delta_j) $.

Aplicando a fórmula [(17)](#equation-eq-40) da proposição para avaliar [(15)](#equation-eq-38) com $ g (z) = c (z) $ e $ h (z ^ {- 1}) = 1 - \delta z ^ {- 1} $ dá

$$
b(L)=\left[{Lc(L)-\delta c(\delta)\over L-\delta}\right] c(L)^{-1}
$$

ou

$$
b(L) =
\left[
    {1-\delta c (\delta) L^{-1} c (L)^{-1}\over 1-\delta L^{-1}}
\right]
$$

Assim, temos:


<a id='equation-eq-41'></a>
$$
\mathbb{\hat E}
\left[
    \sum^\infty_{j=0} \delta^j X_{t+j}\vert X_t,\, x_{t-1},
    \ldots
\right]  =
\left[
    {1-\delta c (\delta) L^{-1} c(L)^{-1} \over 1 - \delta L^{-1}}
\right]
\, X_t \tag{18}
$$

Essa fórmula é útil na solução de versões estocásticas do problema 1 da aula [Controle clássico com álgebra linear](https://julia.quantecon.org/lu_tricks.html), na qual a aleatoriedade surge porque $ \{a_t \} $ é um estocástico
processo.

O problema é maximizar:


<a id='equation-eq-42'></a>
$$
\mathbb{E}_0
\lim_{N \rightarrow \infty}\
\sum^N_{t-0} \beta^t
\left[
    a_t\, y_t - {1 \over 2}\ hy^2_t-{1 \over 2}\ [d(L)y_t]^2
\right] \tag{19}
$$

onde $ \mathbb {E} _t $ é uma expectativa matemática condicionada às informações
conhecido em $ t $ e onde $ \{a_t \} $ é uma covariância
processo estocástico estacionário com representação da média móvel Wold

$$
a_t = c(L)\, \eta_t
$$

onde

$$
c(L) = \sum^{\tilde n}_{j=0} c_j L^j
$$

e

$$
\eta_t =
a_t - \mathbb{\hat E} [a_t \vert a_{t-1}, \ldots]
$$

O problema é maximizar [(19)](#equation-eq-42) em relação a um plano de contingência
expressar $ y_t $ em função de informações conhecidas em $ t $,
que se supõe ser:
$ (y_ {t-1}, \ y_ {t-2}, \ldots, a_t, \ a_ {t-1}, \ldots) $.

A solução desse problema pode ser alcançada em duas etapas.

Primeiro, ignorando a incerteza, podemos resolver o problema assumindo que $ \{a_t \} $ seja uma sequência conhecida.

A solução é, de cima,

$$
c(L) y_t = c(\beta L^{-1})^{-1} a_t
$$

ou


<a id='equation-eq-43'></a>
$$
(1-\lambda_1 L) \ldots (1 - \lambda_m L) y_t
= \sum^m_{j=1} A_j
\sum^\infty_{k=0} (\lambda_j \beta)^k\, a_{t+k} \tag{20}
$$

Segundo, a solução do problema sob incerteza é obtida por
substituindo os termos no lado direito das expressões acima por
seus preditores lineares de mínimos quadrados.

Usando [(18)](#equation-eq-41) e [(20)](#equation-eq-43), temos
a seguinte solução:

$$
(1-\lambda_1 L) \ldots (1-\lambda_m L) y_t
=
\sum^m_{j=1} A_j
 \left[
     \frac{1-\beta \lambda_j \, c (\beta \lambda_j) L^{-1} c(L)^{-1} }
     { 1-\beta \lambda_j L^{-1} }
 \right] a_t
$$

## Predição de Dimensões Finitas

Seja $ (x_1, x_2, \ldots, x_T) ^ \prime = x $ um vetor $ T \times 1 $ de variáveis aleatórias com média $ \mathbb {E} x = 0 $ e matriz de covariância $ \mathbb {E } xx ^ \prime = V $.

Aqui $ V $ é uma matriz definida positiva de $ T \times T $.

Consideraremos as variáveis aleatórias como sendo
ordenado no tempo, para que $ x_t $ seja considerado o valor de alguns
variável econômica no momento $ t $.

Por exemplo, $ x_t $ pode ser gerado pelo processo aleatório descrito pela representação Wold apresentada na equação [(8)](#equation-eq-31).

Nesse caso, $ V_ {ij} $ é dado pelo coeficiente em $ z ^ {\mid ij \mid} $ na expansão de $ g_x (z) = d (z) \, d (z ^ {- 1 }) + h $, que é igual a
$ h + \sum ^ \infty_ {k = 0} d_k d_ {k + \mid i-j \mid} $.

Estaremos interessados em construir preditores lineares de mínimos quadrados para $ j $

$$
\mathbb{\hat E}
\left[
    x_T\vert x_{T-j}, x_{T-j + 1}, \ldots, x_1
\right]
$$

onde $ \mathbb {\hat E} $ é o operador de projeção de mínimos quadrados lineares.

A solução deste problema pode ser exibida através da construção de um
base ortonormal de variáveis aleatórias $ \varepsilon $ por $ x $.

Como $ V $ é um valor positivo definido e simétrico, sabemos que
existe uma decomposição (Cholesky) de $ V $ tal que:

$$
V = L^{-1} (L^{-1})^\prime
$$

ou

$$
L \, V \, L^\prime = I
$$

onde $ L $ é retangular menor e, portanto, também é $ L ^ {- 1} $.

Forme a variável aleatória $ Lx = \varepsilon $.

Então $ \varepsilon $ é uma base ortonormal para $ x $, pois $ L $ é não singular e $ \mathbb {E} \, \varepsilon \, \varepsilon ^ \prime =
L \mathbb {E} xx ^ \prime L ^ \prime = I $.

É conveniente escrever as equações $ Lx = \varepsilon $ e $ L ^ {- 1} \varepsilon = x $


<a id='equation-eq-53'></a>
$$
\begin{aligned}
    L_{11} x_1 &= \varepsilon_1 \\
    L_{21}x_1 + L_{22} x_2 &= \varepsilon_2 \\ \, \vdots \\
    L_{T1} \, x_1 \, \ldots \, + L_{TTx_T} &= \varepsilon_T
\end{aligned} \tag{21}
$$

ou


<a id='equation-eq-54'></a>
$$
\sum^{t-1}_{j=0} L_{t,t-j}\, x_{t-j} = \varepsilon_t, \quad t = 1, \, 2, \ldots T \tag{22}
$$

Nós temos também:


<a id='equation-eq-55'></a>
$$
x_t = \sum^{t-1}_{j=0} L^{-1}_{t,t-j}\, \varepsilon_{t-j}\ . \tag{23}
$$

Observe em [(23)](#equation-eq-55) que $ x_t $ está no espaço medido por
$ \varepsilon_t, \, \varepsilon_ {t-1}, \ldots, \varepsilon_1 $ e de [(22)](#equation-eq-54) que
$ \varepsilon_t $ está no espaço estendido por $ x_t, \, x_ {t-1}, \ldots, \, x_1 $.

Portanto, temos isso para $ t-1 \geq m \geq 1 $


<a id='equation-eq-56'></a>
$$
\mathbb{\hat E}
[ x_t \mid x_{t-m},\, x_{t-m-1}, \ldots, x_1 ] =
\mathbb{\hat E}
[x_t \mid \varepsilon_{t-m}, \varepsilon_{t-m-1},\ldots, \varepsilon_1] \tag{24}
$$

Para $ t-1 \geq m \geq 1 $ reescreva [(23)](#equation-eq-55) como:


<a id='equation-eq-57'></a>
$$
x_t = \sum^{m-1}_{j=0} L_{t,t-j}^{-1}\, \varepsilon_{t-j} + \sum^{t-1}_{j=m}
L^{-1}_{t, t-j}\, \varepsilon_{t-j} \tag{25}
$$

A representação [(25)](#equation-eq-57) é uma decomposição ortogonal de $ x_t $ em uma parte $ \sum ^ {t-1} _ {j = m} L_ {t, tj} ^ {- 1 } \, \varepsilon_ {tj} $ que se encontra no espaço ocupado por
$ [x_ {t-m}, \, x_ {t-m + 1}, \, \ldots, x_1] $ e uma ortogonal
componente não neste espaço.

### Implementação

O código que calcula soluções para problemas de controle e filtragem de LQ usando os métodos descritos aqui e em [Controle clássico com álgebra linear](https://julia.quantecon.org/lu_tricks.html) pode ser encontrado no arquivo [control_and_filter.jl](https://github.com/QuantEcon/QuantEcon.lectures.code/blob/master/lu_tricks/control_and_filter.jl).

Aqui está como fica:

In [3]:
using Polynomials, LinearAlgebra

function LQFilter(d, h, y_m;
                  r = nothing,
                  β = nothing,
                  h_eps = nothing)

    m = length(d) - 1
    m == length(y_m) ||
        throw(ArgumentError("y_m and d must be of same length = $m"))

    # definir os coeficientes do front up ϕ
    ϕ = zeros(2m + 1)
    for i in -m:m
        ϕ[m-i+1] = sum(diag(d*d', -i))
    end
    ϕ[m+1] = ϕ[m+1] + h

    # se r é dado, calcule o vetor ϕ_r
    if isnothing(r)
        k = nothing
        ϕ_r = nothing
    else
        k = size(r, 1) - 1
        ϕ_r = zeros(2k + 1)

        for i = -k:k
            ϕ_r[k-i+1] = sum(diag(r*r', -i))
        end

        if isnothing(h_eps) == false
            ϕ_r[k+1] = ϕ_r[k+1] + h_eps
        end
    end

    # se β é dado, definir as variáveis transformadas
    if isnothing(β)
        β = 1.0
    else
        d = β.^(collect(0:m)/2) * d
        y_m = y_m * β.^(- collect(1:m)/2)
    end

    return (d = d, h = h, y_m = y_m, m = m, ϕ = ϕ, β = β,
            ϕ_r = ϕ_r, k = k)
end

function construct_W_and_Wm(lqf, N)

    d, m = lqf.d, lqf.m

    W = zeros(N + 1, N + 1)
    W_m = zeros(N + 1, m)

    # condições do terminal
    D_m1 = zeros(m + 1, m + 1)
    M = zeros(m + 1, m)

    # (1) construir a matriz D_{m+1} usando a fórmula

    for j in 1:(m+1)
        for k in j:(m+1)
            D_m1[j, k] = dot(d[1:j, 1], d[k-j+1:k, 1])
        end
    end

    # Fazer a matriz simétrica
    D_m1 = D_m1 + D_m1' - Diagonal(diag(D_m1))

    # (2) Construir a matriz M usando as entradas de D_m1

    for j in 1:m
        for i in (j + 1):(m + 1)
            M[i, j] = D_m1[i-j, m+1]
        end
    end
    M

    # Equações de Euler para t = 0, 1, ..., N-(m+1)
    ϕ, h = lqf.ϕ, lqf.h

    W[1:(m + 1), 1:(m + 1)] = D_m1 + h * I
    W[1:(m + 1), (m + 2):(2m + 1)] = M

    for (i, row) in enumerate((m + 2):(N + 1 - m))
        W[row, (i + 1):(2m + 1 + i)] = ϕ'
    end

    for i in 1:m
        W[N - m + i + 1 , end-(2m + 1 - i)+1:end] = ϕ[1:end-i]
    end

    for i in 1:m
        W_m[N - i + 2, 1:(m - i)+1] = ϕ[(m + 1 + i):end]
    end

    return W, W_m
end

function roots_of_characteristic(lqf)
    m, ϕ = lqf.m, lqf.ϕ

    # Calcular as raízes do polinômio 2m
    ϕ_poly = Poly(ϕ[end:-1:1])
    proots = roots(ϕ_poly)
    # classifique as raízes de acordo com seu comprimento (em ordem decrescente)
    roots_sorted = sort(proots, by=abs)[end:-1:1]
    z_0 = sum(ϕ) / polyval(poly(proots), 1.0)
    z_1_to_m = roots_sorted[1:m]     # precisamos apenas daqueles fora do círculo unitário
    λ = 1 ./ z_1_to_m
    return z_1_to_m, z_0, λ
end

function coeffs_of_c(lqf)
    m = lqf.m
    z_1_to_m, z_0, λ = roots_of_characteristic(lqf)
    c_0 = (z_0 * prod(z_1_to_m) * (-1.0)^m)^(0.5)
    c_coeffs = coeffs(poly(z_1_to_m)) * z_0 / c_0
    return c_coeffs
end

function solution(lqf)
    z_1_to_m, z_0, λ = roots_of_characteristic(lqf)
    c_0 = coeffs_of_c(lqf)[end]
    A = zeros(lqf.m)
    for j in 1:m
        denom = 1 - λ/λ[j]
        A[j] = c_0^(-2) / prod(denom[1:m .!= j])
    end
    return λ, A
end

function construct_V(lqf; N = nothing)
    if isnothing(N)
        error("N must be provided!!")
    end
    if !(N isa Integer)
        throw(ArgumentError("N must be Integer!"))
    end

    ϕ_r, k = lqf.ϕ_r, lqf.k
    V = zeros(N, N)
    for i in 1:N
        for j in 1:N
            if abs(i-j) ≤ k
                V[i, j] = ϕ_r[k + abs(i-j)+1]
            end
        end
    end
    return V
end

function simulate_a(lqf, N)
    V = construct_V(N + 1)
    d = MVNSampler(zeros(N + 1), V)
    return rand(d)
end

function predict(lqf, a_hist, t)
    N = length(a_hist) - 1
    V = construct_V(N + 1)

    aux_matrix = zeros(N + 1, N + 1)
    aux_matrix[1:t+1 , 1:t+1 ] = Matrix(I, t + 1, t + 1)
    L = cholesky(V).U'
    Ea_hist = inv(L) * aux_matrix * L * a_hist

    return Ea_hist
end

function optimal_y(lqf, a_hist, t = nothing)
    β, y_m, m = lqf.β, lqf.y_m, lqf.m

    N = length(a_hist) - 1
    W, W_m = construct_W_and_Wm(lqf, N)

    F = lu(W, Val(true))

    L, U = F
    D = diagm(0 => 1.0 ./ diag(U))
    U = D * U
    L = L * diagm(0 => 1.0 ./ diag(D))

    J = reverse(Matrix(I, N + 1, N + 1), dims = 2)

    if isnothing(t)                      # se o problema é determinístico
        a_hist = J * a_hist

        # transforme a sequência a se β é dado
        if β != 1
            a_hist =  reshape(a_hist * (β^(collect(N:0)/ 2)), N + 1, 1)
        end

        ā = a_hist - W_m * y_m        # ā da aula
        Uy = \(L, ā)                  # U @ ȳ = L^{-1}ā da aula
        ȳ = \(U, Uy)                  # ȳ = U^{-1}L^{-1}ā
        # Reverter a ordem de ȳ com a matriz J
        J = reverse(Matrix(I, N + m + 1, N + m + 1), dims = 2)
        y_hist = J * vcat(ȳ, y_m)     # y_hist : concatenados y_m e ȳ
        # transforme de volta na sequência ótima se β é dado
        if β != 1
            y_hist = y_hist .* β.^(- collect(-m:N)/2)
        end

    else                                  # se o problema é estocástico e olhamos para ele
        Ea_hist = reshape(predict(a_hist, t), N + 1, 1)
        Ea_hist = J * Ea_hist

        ā = Ea_hist - W_m * y_m       # ā da aula
        Uy = \(L, ā)                  # U @ ȳ = L^{-1}ā da aula
        ȳ = \(U, Uy)                  # ȳ = U^{-1}L^{-1}ā

        # Reverta a ordem de ȳ com a matriz J
        J = reverse(Matrix(I, N + m + 1, N + m + 1), dims = 2)
        y_hist = J * vcat(ȳ, y_m)     # y_hist : concatenar y_m e ȳ
    end
    return y_hist, L, U, ȳ
end

optimal_y (generic function with 2 methods)

Vamos usar esse código para abordar dois exemplos interessantes.

### Exemplo 1

Considere um processo estocástico com representação da média móvel

$$
x_t = (1 - 2 L) \varepsilon_t
$$

onde $ \varepsilon_t $ é um processo aleatório não correlacionado em série com zero médio e unidade de variância.

Queremos usar a fórmula de Wiener-Kolmogorov [(13)](#equation-eq-36) para calcular as previsões lineares de mínimos quadrados $ \mathbb {E} [x_ {t + j} \mid x_t, x_ {t- 1}, \ldots] $, para $ j = 1, \, 2 $.

Podemos fazer tudo o que queremos definindo $ d = r $, gerando uma instância do `LQFilter` e, em seguida, invocando métodos pertinentes do `LQFilter`.

In [4]:
m = 1
y_m = zeros(m)
d = [1.0, -2.0]
r = [1.0, -2.0]
h = 0.0
example = LQFilter(d, h, y_m, r=d)

(d = [1.0, -2.0], h = 0.0, y_m = [0.0], m = 1, ϕ = [-2.0, 5.0, -2.0], β = 1.0, ϕ_r = [-2.0, 5.0, -2.0], k = 1)

A representação Wold é calculada por example.coefficients_of_c ().

Vamos verificar se "vira raízes", conforme necessário:

In [5]:
coeffs_of_c(example)

2-element Array{Float64,1}:
  2.0
 -1.0

In [6]:
roots_of_characteristic(example)

([2.0], -2.0, [0.5])

Agora vamos formar a matriz de covariância de um vetor de série temporal de comprimento $ N $
e coloque em $ V $.

Então, tomaremos uma decomposição de Cholesky de $ V = L ^ {- 1} L ^ {- 1} = Li Li '$ e a usaremos para formar o vetor de “representações médias móveis” $ x = Li \varepsilon $ e o vetor de "representações autoregressivas" $ L x = \varepsilon $

In [7]:
V = construct_V(example,N=5)

5×5 Array{Float64,2}:
  5.0  -2.0   0.0   0.0   0.0
 -2.0   5.0  -2.0   0.0   0.0
  0.0  -2.0   5.0  -2.0   0.0
  0.0   0.0  -2.0   5.0  -2.0
  0.0   0.0   0.0  -2.0   5.0

Observe como as linhas inferiores das "representações da média móvel" estão convergindo para o histórico infinito apropriado.

In [8]:
F = cholesky(V)
Li = F.L

5×5 LowerTriangular{Float64,Array{Float64,2}}:
  2.23607     ⋅         ⋅         ⋅         ⋅     
 -0.894427   2.04939    ⋅         ⋅         ⋅     
  0.0       -0.9759    2.01187    ⋅         ⋅     
  0.0        0.0      -0.9941    2.00294    ⋅     
  0.0        0.0       0.0      -0.998533  2.00073

Observe como as linhas inferiores das "representações autoregressivas" estão convergindo para a representação autoregressiva apropriada do histórico infinito.

In [9]:
L = inv(Li)

5×5 LowerTriangular{Float64,Array{Float64,2}}:
 0.447214    ⋅          ⋅         ⋅         ⋅      
 0.19518    0.48795     ⋅         ⋅         ⋅      
 0.0946762  0.236691   0.49705    ⋅         ⋅      
 0.0469898  0.117474   0.246696  0.499266   ⋅      
 0.0234518  0.0586295  0.123122  0.249176  0.499817

**Observação** Seja $ \pi (z) = \sum ^ m_ {j = 0} \pi_j z ^ j $ e deixe $ z_1, \ldots,
z_k $ são os zeros de $ \pi (z) $ que estão dentro do círculo unitário, $ k <m $.

Então defina:

$$
\theta (z) = \pi (z) \Biggl( {(z_1 z-1) \over (z-z_1)} \Biggr)
\Biggl( { (z_2 z-1) \over (z-z_2) } \Biggr ) \ldots \Biggl({(z_kz-1) \over
(z-z_k) }\Biggr)
$$

O termo multiplicando $ \pi (z) $ é denominado “fator de Blaschke”.

Então pode ser provado diretamente que:

$$
\theta (z^{-1}) \theta (z) = \pi (z^{-1}) \pi (z)
$$

e que os zeros de $ \theta (z) $ não estão dentro do círculo unitário.

### Exemplo 2

Considere um processo estocástico $ X_t $ com média móvel
representada por:

$$
X_t = (1 - \sqrt 2 L^2) \varepsilon_t
$$

onde $ \varepsilon_t $ é um processo aleatório não correlacionado em série
com zero médio e unidade de variância.

Vamos encontrar uma representação da média móvel Wold por $ x_t $.

Vamos usar a fórmula de Wiener-Kolomogorov [(13)](#equation-eq-36) para calcular as previsões lineares de mínimos quadrados
$ \mathbb {\hat E} \left [X_ {t + j} \mid X_ {t-1}, \ldots \right] \hbox {para} j = 1, \, 2, \, 3 $.

Prosseguimos da mesma maneira que no exemplo 1:

In [10]:
m = 2
y_m = [0.0, 0.0]
d = [1, 0, -sqrt(2)]
r = [1, 0, -sqrt(2)]
h = 0.0
example = LQFilter(d, h, y_m, r = d)

(d = [1.0, 0.0, -1.4142135623730951], h = 0.0, y_m = [0.0, 0.0], m = 2, ϕ = [-1.4142135623730951, 0.0, 3.0000000000000004, 0.0, -1.4142135623730951], β = 1.0, ϕ_r = [-1.4142135623730951, 0.0, 3.0000000000000004, 0.0, -1.4142135623730951], k = 2)

In [11]:
coeffs_of_c(example)

3-element Array{Float64,1}:
  1.4142135623731003
 -0.0               
 -1.0000000000000064

In [12]:
roots_of_characteristic(example)

([1.1892071150027195, -1.1892071150027195], -1.4142135623731096, [0.8408964152537157, -0.8408964152537157])

In [13]:
V = construct_V(example, N = 8)

8×8 Array{Float64,2}:
  3.0       0.0      -1.41421   0.0      …   0.0       0.0       0.0    
  0.0       3.0       0.0      -1.41421      0.0       0.0       0.0    
 -1.41421   0.0       3.0       0.0          0.0       0.0       0.0    
  0.0      -1.41421   0.0       3.0         -1.41421   0.0       0.0    
  0.0       0.0      -1.41421   0.0          0.0      -1.41421   0.0    
  0.0       0.0       0.0      -1.41421  …   3.0       0.0      -1.41421
  0.0       0.0       0.0       0.0          0.0       3.0       0.0    
  0.0       0.0       0.0       0.0         -1.41421   0.0       3.0    

In [14]:
F = cholesky(V)
Li = F.L
Li[end-2:end, :]

3×8 Array{Float64,2}:
 0.0  0.0  0.0  -0.92582   0.0        1.46385   0.0      0.0    
 0.0  0.0  0.0   0.0      -0.966092   0.0       1.43759  0.0    
 0.0  0.0  0.0   0.0       0.0       -0.966092  0.0      1.43759

In [15]:
L = inv(Li)

8×8 LowerTriangular{Float64,Array{Float64,2}}:
 0.57735    ⋅         ⋅         ⋅        …   ⋅         ⋅         ⋅      
 0.0       0.57735    ⋅         ⋅            ⋅         ⋅         ⋅      
 0.308607  0.0       0.654654   ⋅            ⋅         ⋅         ⋅      
 0.0       0.308607  0.0       0.654654      ⋅         ⋅         ⋅      
 0.19518   0.0       0.414039  0.0           ⋅         ⋅         ⋅      
 0.0       0.19518   0.0       0.414039  …  0.68313    ⋅         ⋅      
 0.131165  0.0       0.278243  0.0          0.0       0.695608   ⋅      
 0.0       0.131165  0.0       0.278243     0.459078  0.0       0.695608

### Predição

Segue-se imediatamente do “princípio da ortogonalidade” dos mínimos quadrados (consulte [AP91](https://julia.quantecon.org/../zreferences.html#athanasios1991) ou [Sar87](https://julia.quantecon.org/../zreferences.html#sargent1987) [cap. X]) que:


<a id='equation-eq-58'></a>
$$
\begin{aligned}
    \mathbb{\hat E} & [x_t \mid x_{t-m},\, x_{t-m+1}, \ldots x_1]
                    = \sum^{t-1}_{j=m} L^{-1}_{t,t-j}\, \varepsilon_{t-j} \\
               & = [L_{t, 1}^{-1}\, L^{-1}_{t,2},\, \ldots, L^{-1}_{t,t-m}\ 0 \ 0 \ldots 0] L \, x
\end{aligned} \tag{26}
$$

Isso pode ser interpretado como uma versão de dimensão finita da fórmula de previsão Wiener-Kolmogorov $ m $ -passos a frente.

Podemos usar [(26)](#equation-eq-58) para representar a projeção linear de mínimos quadrados de
o vetor $ x $ condicionado nas primeiras observações $ s $
$ [x_s, x_ {s-1} \ldots, x_1] $.

Nós temos


<a id='equation-eq-59'></a>
$$
\mathbb{\hat E}[x \mid x_s, x_{s-1}, \ldots, x_1]
= L^{-1}
\left[
    \begin{matrix}
        I_s & 0 \\
        0 & 0_{(t-s)}
    \end{matrix}
\right] L x \tag{27}
$$

Essa fórmula será conveniente para representar a solução dos problemas de controle sob incerteza.

A equação [(23)](#equation-eq-55) pode ser reconhecida como uma versão dimensional finita de uma representação de média móvel.

A equação [(22)](#equation-eq-54) pode ser vista como uma versão de dimensão finita de uma representação auto-regressiva.

Observe que mesmo,
se o processo $ x_t $ for covariância estacionária, para que $ V $
é tal que $ V_ {ij} $ depende apenas de $ \vert i-j \vert $, o
coeficientes na representação da média móvel são dependentes do tempo,
havendo uma média móvel diferente para cada $ t $.

E se
$ x_t $ é um processo estacionário de covariância, a última linha de
$ L ^ {- 1} $ converge para os coeficientes na média móvel Wold
representação para $ \{x_t \} $ como $ T \rightarrow \infty $.

Além disso, se $ x_t $ é covariância estacionária, para $ k $ fixo
e $ j> 0, \, L ^ {- 1} _ {T, T-j} $ convergem para
$ L ^ {- 1} _ {T-k, T-k-j} $ como $ T \rightarrow \infty $.

Isso é,
as linhas "inferiores" de $ L ^ {- 1} $ convergem entre si e para o
Coeficientes médios móveis de Wold como $ T \rightarrow \infty $.

Esta última observação fornece uma maneira prática simples e amplamente usada de
formando uma aproximação finita de $ T $ a uma média móvel de Wold
representação.

Primeiro, forme a matriz de covariância
$ \mathbb {E} xx ^ \prime = V $ e obtenha a decomposição de Cholesky
$ L ^ {- 1} L ^ {- 1 ^ \prime} $ de $ V $, o que pode ser realizado
rapidamente em um computador.

A última linha de $ L ^ {- 1} $ fornece os coeficientes médios móveis aproximados de Wold.

Este método pode ser facilmente generalizado para sistemas multivariados.


<a id='fdcp'></a>

## Controle Dimensional Finito Combinado e Previsão

Considere o problema de controle de dimensão finita, maximize:

$$
\mathbb{E} \, \sum^N_{t=0} \,
\left\{
     a_t y_t - {1 \over 2} h y^2_t - {1 \over 2} [d(L) y_t ]^2
\right\},\  \quad h > 0
$$

onde $ d (L) = d_0 + d_1 L + \ldots + d_m L ^ m $, $ L $ é o
operador lag, $ \bar a = [a_N, a_ {N-1} \ldots, a_1, a_0] ^ \prime $ a
vetor aleatório com média zero e $ \mathbb {E} \, \bar a \bar a ^ \prime = V $.

As variáveis $ y _ {- 1}, \ldots, y _ {- m} $ são fornecidas.

A maximização está acima das opções de $ y_0,
y_1 \ldots, y_N $, em que $ y_t $ é necessário para ser uma função linear
de $ \{y_ {t-s-1}, t + m-1 \geq 0; \ a_ {t-s}, t \geq s \geq 0 \} $.

Vimos na palestra [Controle clássico com álgebra linear](https://julia.quantecon.org/lu_tricks.html) que a solução desse problema, com certeza, poderia ser representada na forma feedback-feedforward:

$$
U \bar y
   = L^{-1}\bar a + K
   \left[
     \begin{matrix}
         y_{-1}\\
         \vdots\\
         y_{-m}
     \end{matrix}
   \right]
$$

por alguns $ (N + 1) \times m $ matriz $ K $.

Usando uma versão da fórmula [(26)](#equation-eq-58), podemos expressar $ \mathbb {\hat E} [\bar a \mid a_s, \, a_ {s-1}, \ldots, a_0] $ como:

$$
\mathbb{\hat E}
[ \bar a \mid a_s,\, a_{s-1}, \ldots, a_0]
= \tilde U^{-1}
\left[
    \begin{matrix}
        0 & 0 \\
        0 & I_{(s+1)}
    \end{matrix}
\right]
\tilde U \bar a
$$

onde $ I _ {(s + 1)} $ é a  $ (s + 1) \times (s + 1) $
matriz identidade e $ V = \tilde U^{- 1} \tilde U^{-1^{\prime}} $, em que
$ \tilde U $ é o fator Cholesky retangular *superior* da
matriz de covariância $ V $.

(Revertemos o eixo do tempo ao datar os $ a $ em relação ao anterior)

O eixo do tempo pode ser revertido na representação [(27)](#equation-eq-59) substituindo $ L $ por $ L ^ T $.

A regra de decisão ideal para usar no momento $ 0 \leq t \leq N $ é então
fornecido pela linha $ (N-t +1) ^ {\rm th} $ de:

$$
U \bar y = L^{-1} \tilde U^{-1}
    \left[
        \begin{matrix}
            0 & 0 \\
            0 & I_{(t+1)}
        \end{matrix}
    \right]
    \tilde U \bar a + K
    \left[
    \begin{matrix}
        y_{-1}\\
        \vdots\\
        y_{-m}
    \end{matrix}
    \right]
$$

## Exercícios

### Exercício 1

Seja $ Y_t = (1 - 2 L) u_t $ onde $ u_t $ é um ruído branco
de média nula com $ \mathbb {E} u ^ 2_t = 1 $. Seja:

$$
X_t = Y_t + \varepsilon_t
$$

onde $ \varepsilon_t $ é um ruído branco não correlacionado em série com
$ \mathbb {E} \varepsilon ^ 2_t = 9 $ e $ \mathbb {E} \varepsilon_t u_s = 0 $ para todos
$ t $ e $ s $.

Encontre a representação da média móvel Wold por $ X_t $.

Encontre uma fórmula para os $ A_ {1j} $ 's em:

$$
\mathbb{E} \widehat X_{t+1} \mid X_t, X_{t-1}, \ldots = \sum^\infty_{j=0} A_{1j}
X_{t-j}
$$

Encontre uma fórmula para os $ A_ {2j} $ 's em:

$$
\mathbb{\hat E} X_{t+2} \mid X_t, X_{t-1}, \ldots = \sum^\infty_{j=0} A_{2j}
X_{t-j}
$$

### Exercício 2

(Previsão multivariável) Seja $ Y_t $ um $ (n \times 1) $
processo estocástico de vetor com representação de média móvel:

$$
Y_t = D(L) U_t
$$

onde $ D (L) = \sum ^ m_ {j = 0} D_j L ^ J, D_j $ uma $ n \times n $
matriz, $ U_t $ um $ (n \times 1) $ vetor de ruído branco  com
:$ \mathbb: \mathbb {E} (U_t) = 0 $ para todos os $ t $, $ \mathbb {E} (U_t U_s ')= 0 $ para todos os $ s \neq t $,
e $ \mathbb {E} (U_t U_t ')= I $ para todos os $ t $.

Seja $ \varepsilon_t $ vetor de ruido branco $ n \times 1 $ com média $ 0 $ e matriz de covariância contemporânea $ H $, onde $ H $ é uma matriz definida positiva.

Deixe $ X_t = Y_t + \varepsilon_t $.

Defina os covariograms como $ C_X
(\tau) = \mathbb {E} (X_t X ^ \prime_ {t- \tau}, C_Y tau) = \mathbb {E} Y_t Y ^ \prime_ {t- \tau},
C_ {YX} (\tau) = \mathbb {E} Y_t X ^ \prime_ {t- \tau} $.

Em seguida, defina a matriz
função geradora de covariância, como em [(21)](https://julia.quantecon.org/lu_tricks.html#equation-onetwenty), apenas interprete todas as
objetos em [(21)](https://julia.quantecon.org/lu_tricks.html#equation-onetwenty) como matrizes.

Mostre que as funções geradoras de covariância são dadas por:

$$
\begin{aligned}
    g_y (z) &= D (z) D (z^{-1})^\prime \\
    g_X (z) &= D (z) D (z^{-1})^\prime + H \\
    g_{YX} (z) &= D (z) D (z^{-1})^\prime
\end{aligned}
$$

Uma fatoração de $ g_X (z) $ pode ser encontrada (consulte [Roz67](https://julia.quantecon.org/../zreferences.html#rozanov1967) ou [Whi83](https://julia.quantecon.org/../zreferences.html#whittle1983)) do formulário:

$$
D (z) D (z^{-1})^\prime + H = C (z) C (z^{-1})^\prime, \quad C (z) =
\sum^m_{j=0} C_j z^j
$$

onde os zeros de $ \vert C (z) \vert $ não estão dentro da unidade
círculo.

Uma representação média móvel de Wold vetorial de $ X_t $ é então:

$$
X_t = C(L) \eta_t
$$

onde $ \eta_t $ é um ruído branco de vetor $ (n \times $ que
é "fundamental" para $ X_t $.

Ou seja, $ X_t - \mathbb {\hat E} \left [X_t \mid X_ {t-1}, X_ {t-2}
\ldots \right] = C_0 \, \eta_t $.

O preditor ideal de $ X_ {t + j} $ é

$$
\mathbb{\hat E} \left[X_{t+j} \mid X_t, X_{t-1}, \ldots\right]
 = \left[{C(L) \over L^j} \right]_+ \eta_t
$$

Se $ C (L) $ for invertível, ou seja, se os zeros de $ \det $
$ C (z) $ estão estritamente fora do círculo unitário, então esta fórmula pode
ser escrita como:

$$
\mathbb{\hat E} \left[X_{t+j} \mid X_t, X_{t-1}, \ldots\right]
    = \left[{C(L) \over L^J} \right]_+ C(L)^{-1}\, X_t
$$